In [1]:
import pandas as pd
import numpy as np

# Massey's Ratings and Rankings 

Adapted from: https://maherou.github.io/Teaching/files/CS317/masseyMethod.pdf

One of the methods currently used by the Bowl Championship Series, Massey's Rating & Rankings system determines which teams in NCAA college football play in each of the bowl games.

This method is based on the theory of least squares and incorporates the number of games played by each
team as well as the difference in points scored by teams that played against each other.

In this example there are five teams:
- John Hopkins
- F&M
- Gettsyburg
- Dickinson
- McDaniel

The win/loss table below shows the outcomes of previous matches. The left-most score belongs to the team in row, the right-most score to the team in column.

In [37]:
data = {'John Hopkins':['-','14-12','35-49','0-49','7-49'],
       'F&M':['12-14','-','38-31','28-36','10-35'],
       'Gettysburg':['49-35','31-38','-','23-13','3-35'],
       'Dickinson':['49-0','36-28','13-23','-','31-38'],
       'McDaniel':['49-7','35-10','35-3','38-31','-']}
df = pd.DataFrame(data)
df.index = df.columns
df

,John Hopkins,F&M,Gettysburg,Dickinson,McDaniel
John Hopkins,-,12-14,49-35,49-0,49-7
F&M,14-12,-,31-38,36-28,35-10
Gettysburg,35-49,38-31,-,13-23,35-3
Dickinson,0-49,28-36,23-13,-,38-31
McDaniel,7-49,10-35,3-35,31-38,-


We can see that each team played every other team once, for a total of four games played by every team.

We can calculate the point differential between two teams that played a game against each other by subtracting one team’s score from their opponent’s score.

We want to calculate the total point differential for each team; that is, for each team, we want to sum the point differences of every game that team played.

Let us start by calculating the total point differential for each team.

In [38]:
differentials = []
# Basically a quick way to sum the differentials per row in a nested list comprehension
for i,row in df.iterrows():
    differentials.append(sum([int(x.split('-')[0])-int(x.split('-')[1]) for x in row.values.tolist() if x != '-']))
# adding to the df    
df['Differentials'] = differentials
df

,John Hopkins,F&M,Gettysburg,Dickinson,McDaniel,Differentials
John Hopkins,-,12-14,49-35,49-0,49-7,103
F&M,14-12,-,31-38,36-28,35-10,28
Gettysburg,35-49,38-31,-,13-23,35-3,15
Dickinson,0-49,28-36,23-13,-,38-31,-40
McDaniel,7-49,10-35,3-35,31-38,-,-106


Massey’s Method is based on the least squares solution to a system of equations. 

An important equation to this idea is the following:

<img src="https://i.ibb.co/rQZNSWs/equation.png" alt="ri - rj = y" title="" />

Where ri and rj are the ratings of teams i and j, and y is the point difference in the game played between teams i and j.

Clearly, as we do not have ratings of the teams before using this method, this equation is not used exactly, but Massey employs a similar idea.

If for every game played between two teams i and j we calculate the point differential, we have the equation i - j = s, where i and j are the points earned by teams i and j respectfully, and s is the difference of the score.

Thus, if a total of m games are played between n teams and we construct a similar equation for each game, the result is a system of equations with m equations and n unknowns. From this, we can construct an m × n matrix, (m rows because there are m equations, one for each game, and n columns, one for each team). We call this matrix X.

In each row, for teams i and j that played each other, there is a 1 in the i space and a −1 in the j space, indicating that these teams played each other, with team i defeating team j, and zeros everywhere else. For our small example, we get the following matrix X:

In [39]:
X = {'John Hopkins':[0,0,0,1,1,0,0,-1,1,0],
       'F&M':[0,1,1,0,0,0,0,1,0,-1],
       'Gettysburg':[1,0,0,0,-1,0,-1,0,0,1],
       'Dickinson':[0,-1,0,-1,0,1,1,0,0,0],
       'McDaniel':[-1,0,-1,0,0,-1,0,0,-1,0]}
X = pd.DataFrame(X)
X

,John Hopkins,F&M,Gettysburg,Dickinson,McDaniel
0,0,0,1,0,-1
1,0,1,0,-1,0
2,0,1,0,0,-1
3,1,0,0,-1,0
4,1,0,-1,0,0
5,0,0,0,1,-1
6,0,0,-1,1,0
7,-1,1,0,0,0
8,1,0,0,0,-1
9,0,-1,1,0,0


The first row, for example, corresponds to the game played between Gettysburg and McDaniel. Gettysburg defeated McDaniel, thus the Gettysburg space has a 1 and the McDaniel space has a −1.

To find the rating of each team, then, we might consider the linear equation Xr = y, where X is our matrix, r is an n × 1 vector of the ratings we are trying to find, and y is an m × 1 vector of the point differentials of every game. We write this as follows:

In [40]:
print("X:")
print(X)
print("\nr:")
r = ['rj','rf','rg','rd','rm']
print(*(f'  {x}' for x in r), sep='\n')
print('\ny:')
print(*(f'  {x}' for x in differentials), sep='\n')

X:
   John Hopkins  F&M  Gettysburg  Dickinson  McDaniel
0             0    0           1          0        -1
1             0    1           0         -1         0
2             0    1           0          0        -1
3             1    0           0         -1         0
4             1    0          -1          0         0
5             0    0           0          1        -1
6             0    0          -1          1         0
7            -1    1           0          0         0
8             1    0           0          0        -1
9             0   -1           1          0         0

r:
  rj
  rf
  rg
  rd
  rm

y:
  103
  28
  15
  -40
  -106


We find, however, that because there are more than likely more games played than there are teams, the matrix is overdetermined, meaning that there are more equations than there are unknowns in the system, and it almost certainly does not have a solution.

Because there is no solution, we multiply by the transpose of X on the left on both sides of the equation and instead we consider the normal equations:

<img src="https://i.ibb.co/Pr1NGfj/equation.png" alt="XTXr=XTy" title="" />

We can solve for the vector r in this equation as a good estimate of the ratings.

We call X(Transpose)X a new matrix M and assign the variable p to an n × 1 vector of total point differentials, one for each team, as we calculated for our small example above. 

<img src="https://i.ibb.co/z8GT7Z5/equation.png" alt="XTXr=XTy" title="" />

The matrix M becomes an n × n matrix with the total number of games played by each team along the diagonal; that is, for team i, entry Mii is the total number of games played by team i. The entry Mij for i != j is equal to the negation of the number of games that teams i and j played against each other. We see that the rows and columns of this matrix sum to zero, so are linearly dependent:

In [41]:
XT = np.transpose(X)
M = np.dot(XT,X)
M = pd.DataFrame(M)
M.columns = X.columns
M.index = M.columns
M #matrix M

,John Hopkins,F&M,Gettysburg,Dickinson,McDaniel
John Hopkins,4,-1,-1,-1,-1
F&M,-1,4,-1,-1,-1
Gettysburg,-1,-1,4,-1,-1
Dickinson,-1,-1,-1,4,-1
McDaniel,-1,-1,-1,-1,4


In [42]:
p = differentials # vector p
p

[103, 28, 15, -40, -106]

We encounter one other small problem: after a few games have been played, this matrix has a rank n−1, which means that there are infinite solutions to this system Mr = p.

We are looking for a single solution so we can obtain the rating of each team. To fix this problem, we change the last row of the matrix to a row of all ones and the corresponding entry in the point differential vector to a zero, so that the rank is no longer less than n.

This system can now be solved and we can find the ratings of each team. In our example, the new system is as follows:

In [43]:
M.iloc[-1,:] = 1
M #matrix M

,John Hopkins,F&M,Gettysburg,Dickinson,McDaniel
John Hopkins,4,-1,-1,-1,-1
F&M,-1,4,-1,-1,-1
Gettysburg,-1,-1,4,-1,-1
Dickinson,-1,-1,-1,4,-1
McDaniel,1,1,1,1,1


In [44]:
p[-1] = 0
p # p 

[103, 28, 15, -40, 0]

Now we can solve r using regression.

<img src="https://i.ibb.co/QpMYDjv/equation.png" alt="XTXr=XTy" title="" />


In [47]:
r = np.dot(np.linalg.inv(np.dot(np.transpose(M),M)),np.dot(np.transpose(M),p))
r # ratings

array([ 20.6,   5.6,   3. ,  -8. , -21.2])

In [49]:
df['Ratings'] = r
df

,John Hopkins,F&M,Gettysburg,Dickinson,McDaniel,Differentials,Ratings
John Hopkins,-,12-14,49-35,49-0,49-7,103,20.6
F&M,14-12,-,31-38,36-28,35-10,28,5.6
Gettysburg,35-49,38-31,-,13-23,35-3,15,3.0
Dickinson,0-49,28-36,23-13,-,38-31,-40,-8.0
McDaniel,7-49,10-35,3-35,31-38,-,-106,-21.2


## Advantages
This method does not require that every team plays the same number of games, as other methods do. It also automatically incorporates ties, as it does not utilize whether a team won or lost, but rather the point differential, so ties are already accounted for, unlike other methods that count teams’ wins and losses. 

## Disadvantages
A disadvantage is that we need to alter the matrix M to be able to solve the system